# Note: I have made separate notebook for each question 

## Question 1

## Use the BeautifulSoup package or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe